the goal of this notebook is to remotely load and preview spectrophotometric standards for use with JWST spectra

would like to click a star name and display both an image cutout preview and an interactive spectrum

Things to Not Like
- no master list of standard stars
- no easy way of getting spectral data of standards in catalog (e.g. calspec)
- replace in network path with public version for spectral standard data grab
- have astroquery make clear what the line catalogs are 
- not one service that gives full list of lines (all have different interfaces and different available properties)
- need input from experts to provide contextual info for important and useful strong lines
- have way to overplot lines based on context and groups/sets of lines 
- do not like scrolling back and forth
- combine the 3 html calspec tables into a single FITS file readable by Astropy Table or Pandas 


In [ ]:
import requests
import os
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
from astropy.io import fits
from astropy.wcs import WCS
import astropy.units as u
import astroquery
from astroquery.skyview import SkyView
import holoviews as hv
hv.extension('bokeh')
%matplotlib inline

In [ ]:
def clean_stis(df):
    '''Clean the STIS column in the catalog '''
    stis=[]
    for row in df.itertuples():
        if row.STIS is None or '_stis' not in row.STIS:
            if row.Model is not None and '_stis' in row.Model:
                name = row.Model
            elif '_stis' in row.Name:
                name = row.Name
        else:
            name = row.STIS
        stis.append(name.lower().replace('*', ''))
    df['STIS'] = stis
    
    # fix the SDSS name
    row = df.loc[df['Star_name'].str.contains("SDSS")]
    row.Name = row.B_V
    df.loc[df['Star_name'].str.contains("SDSS")]=row 

    return df

In [ ]:
# read in a clipboard/catalog table of standards
file = 'calspec_catalog.csv'
if os.path.exists(file):
    df = pd.read_csv(file, index_col=None)
    df = df.fillna('None')
else:
    # copy from Table 1 at http://www.stsci.edu/hst/observatory/crds/calspec.html
    df=pd.read_clipboard(sep='[\s]{2,}', header=0)
    df = df.drop(0)
    # format column names 
    df.columns = [c.replace(' ', '_').replace('**', '').replace('-', '_') for c in df.columns]
    # fix STIS column
    df = clean_stis(df)
    # make specfile column
    df['specfile']=df['Name']+df['STIS']+'.fits'
    # write to csv 
    df.to_csv('calspec_catalog.csv', index=None)
    

In [ ]:
df.head()

In [ ]:
#target = '2MASS J00361617+1821104'
target = '10 Lac'
target_root = df.loc[df['Star_name']==target]['Name'].values.tolist()[0]
surveys = ['DSS2 Red', '2MASS-K']

In [ ]:
def get_image(target, surveys=None, download=False, **kwargs):
    ''' Retrieve image data for a given target '''
    
    if not surveys:
        surveys = ['DSS2 Red', '2MASS-K']
    
    try:
        if download:
            files = SkyView.get_images(position=target,survey=surveys, **kwargs)
        else:
            files = SkyView.get_image_list(position=target,survey=surveys, **kwargs)
    except Exception as e:
        print(f'Error retrieving SkyView preview: {e}')
        return None

    # produce the hdu
    if download:
        hdus = files
    else:
        hdus = []
        for file in files:
            h = create_hdu(file)
            hdus.append(h)
    return hdus

In [ ]:
def create_hdu(file):
    ''' create HDU from a bytes remote request content '''
    r = requests.get(file)
    hdu = None
    if r.ok:
        img = BytesIO(r.content)
        hdu = fits.HDUList.fromfile(img)
    else:
        raise RuntimeError(f'Error getting request. Status Code: {r.status_code}')
    return hdu

In [ ]:
# look up the SkyPreview
hdus = get_image(target, surveys=surveys, radius=1*u.arcmin)
if hdus:
    dss, twomass = hdus

In [ ]:
# get wcs
wcs_dss = WCS(dss[0].header)
wcs_twomass = WCS(twomass[0].header)

In [ ]:
# create static preview
ax = plt.subplot(221, projection=wcs_dss)

ax1 = plt.subplot(1,2,1, projection=wcs_twomass)
ax1.imshow(dss[0].data, origin='lower')
ax1.set_title(f'{surveys[0]}: {target}')

# raw pixel plot
ax2 = plt.subplot(1,2,2, projection=wcs2)
ax2.imshow(twomass[0].data, origin='lower')
ax2.set_title(f'{surveys[1]}: {target}')

ax1.figure.set_size_inches((15,15))
ax2.figure.set_size_inches((15,15))

In [ ]:
def find_spectra(df, target=None, caltype=None):
    ''' find relavant spectra from the calspec directory '''

    if target != '*':
        caltype = df.loc[df['Star name']==target]['STIS**'].values.tolist()[0]
        
    path = '/grp/hst/cdbs/calspec/'
    target = target.replace(' ', '').lower()
    files = glob.glob(path+f'*{target}*{caltype}*.fits')
    return files

In [ ]:
def get_specdf(file):
    ''' Read in a spectro standard star file in produce a Pandas DF '''
    name = file.rsplit('/', 1)[1].split('_')[0]
    h = fits.open(file)
    specdf = Table(h[1].data).to_pandas()
    specdf['STARNAME'] = name.upper() 
    return specdf

next goal - make a widget with dropdown menu of standard to select
list of standards

In [ ]:
# get a list of files from the clipboard table
# files=[]
# for row in df:
#     files.extend(find_spectra(target=row.Name))
# files

#
# TODO: generalize this and replace the find_spectra function
#

# get files from table
path = '/grp/hst/cdbs/calspec/'
#files = [path+'{0}{1}.fits'.format(row.Name,row.STIS.replace('*', '')) for row in df[['Name', 'STIS']].itertuples() if row.STIS != 'None']
#files
files = path + df['specfile']

In [ ]:
# get a list of spectral calibration spectra filenames
#files = find_spectra(df, target=target)
files

In [ ]:
def build_master(files):
    ''' build a master pandas dataframe from a list of spec files '''
    master=None
    for file in files:
        tmp_df = get_specdf(file)
        #print(tmp_df['STARNAME'])
        if master is not None:
            master = master.append(tmp_df)
        else:
            master = tmp_df
    return master

In [ ]:
# build a master list of files
master = build_master(files)
master.head()

In [ ]:
# build interactive display widget for all stars in master list
master_macro = hv.Dataset(master, ['STARNAME', 'WAVELENGTH'])
master_macro

In [ ]:
# create water lines overlay and text
lines=[23472.003301137982,
 23326.981220208632,
 23234.492318049546,
 23185.055480707324,
 23114.723654546422,
 23022.387506391973,
 22981.419669019673,
 22905.610364976637,
 22781.239191728437,
 22309.74469086192]
lines = [np.min(lines), np.max(lines)]
#lines=lines[0]
vline = None
for i, line in enumerate(lines):
    wave = line
    text = hv.Text(wave, 2e-14, f" L{i}", halign='left')
    if vline:
        vline *= hv.VLine(wave).opts(color='black')*text
    else:
        vline = hv.VLine(wave).opts(color='black')*text
    
# create line curves
curves = master_macro.to(hv.Curve, 'WAVELENGTH', 'FLUX', groupby='STARNAME')
curves.redim(WAVELENGTH=hv.Dimension('WAVELENGTH', soft_range=(5000, 50000))).redim(FLUX=hv.Dimension('FLUX', soft_range=(0,3e-14))).opts(width=600)*vline
#curves.opts(width=600)

In [ ]:
dss[0].data.shape

In [ ]:
print(dss[0].data)
import numpy as np
data = hv.Dataset((np.arange(300), np.arange(300), dss[0].data), ['y', 'x'], 'Flux')
data.to(hv.Image,['x', 'y'])


In [ ]:
def hv_image(starname):
    # data = dss[0]
    hdu = get_image(starname, surveys=['DSS2 Red'], radius=1*u.arcmin)[0]
    data = hdu[0].data
    n1, n2 = data.shape
    dataset = hv.Dataset((np.arange(n1), np.arange(n2), data), ['y', 'x'], 'Flux')
    return dataset.to(hv.Image, ['x', 'y'])

In [ ]:
image = hv.DynamicMap(hv_image, kdims=['starname'])


In [ ]:
image

In [ ]:
image.redim.range(starname=('10 Lac', 'GD 153'))

In [ ]:
def test(starname):
    hdu = get_image(starname, surveys=['DSS2 Red'], radius=1*u.arcmin)[0]
    return hv.Image(hdu[0].data)

In [ ]:
testim = hv.DynamicMap(test, kdims=['starname'])
testim
testim.redim.range(starname=('10 Lac', 'GD 153'))
#clifford.redim.range(a=(-1.5,-1),b=(1.5,2),c=(1,1.2),d=(0.75,0.8), x=(-2,2), y=(-2,2))


In [ ]:
newdf = pd.read_clipboard('[\s]{2,}')
newdf=newdf.drop(0)
newdf.columns = [c.replace(' ', '_').replace('**', '').replace('-', '_') for c in newdf.columns]
newdf.head()

In [ ]:
files = [path+f for f in df['specfile'].values.tolist()]

In [ ]:
files

In [ ]:
d=get_image(target, surveys=['DSS2 Red'], radius=1*u.arcmin)


In [ ]:
d[0].shape

In [ ]:
df['Star_name']

In [ ]:
subset = master.loc[master['STARNAME']=='10LAC']

In [ ]:
subset

In [ ]:
c=hv.Curve(subset, kdims=['WAVELENGTH', 'FLUX']).opts(width=600).redim(WAVELENGTH=hv.Dimension('WAVELENGTH', range=(0, 50000)))
t=hv.Image(dss[0].data)
e=c+t
e